# Client Analysis
- This notebook is ideal to understand schema and data within the databases storeing in data/

In [1]:
# Import libraries
import sqlite3
import pandas as pd
import json

In [2]:
# Connect to the database
conn = sqlite3.connect('../data/client_1_cardio.db')

In [4]:
# Read tables into DataFrames
workouts = pd.read_sql_query("SELECT * FROM cardio", conn)
sessions = pd.read_sql_query("SELECT * FROM aggregated_cardio_session_data", conn)

# Close connection
conn.close()

print(f"Loaded {len(workouts)} workouts, {len(sessions)} sessions")

Loaded 3 workouts, 108 sessions
